In [1]:
import CodaPCA
import CodaCl 
import PCACl
import numpy as np
from runpca import read_csv
import os
import sklearn
from sklearn.linear_model import Ridge
from sklearn.linear_model import LogisticRegression
#change module for newer sklearn versions
from sklearn.model_selection  import cross_val_score
from sklearn.model_selection  import KFold
import matplotlib.pyplot as plt
import pandas as pd
from sklearn import decomposition
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
import torch
import dill
import importlib
importlib.reload(CodaCl)

<module 'CodaCl' from 'C:\\Users\\u5801283\\Documents\\Honours\\honours_final\\honours\\Code\\coda-pca-orig\\coda\\codes\\CodaCl.py'>

In [2]:
def enhanced_cross_val(features, targets, folds):
    assert len(features) == len(targets), "Mismatch in length of features and targets"
    kfold_scores = []
    for train, test in folds:
        Y_train = targets[train]
        X_train = features[train]
               
        Y_test = targets[test]
        X_test = features[test]
                
        model = LogisticRegression(multi_class='auto', solver='lbfgs')
        model.fit(X_train, Y_train)
        y_pred = model.predict(X_test)
        kfold_scores.append(sklearn.metrics.accuracy_score(Y_test,y_pred))
                
    return kfold_scores

In [130]:
def coda_val(features, targets, n_components, folds, nn_shape,  lr,lam, epochs):
    kfold_scores = []
    for train, test in folds:        
        Y_train = targets[train]
        X_train = features[train]
        
       
        Y_test = targets[test]
        X_test = features[test]
        
        
        model = CodaCl.CoDA_Cl(features.shape[1], n_components, len(np.unique(targets)), nn_shape[0], nn_shape[1])


        val_arr, train_arr = model.fit(X_train, Y_train, lam, lr, train_size = int(len(X_train)*(3/4)), epochs=epochs)
        
        #print ("Plot!")
        
#         plt.plot(val_arr, c="red")
#         plt.plot(train_arr, c="blue")
        
#         plt.ylim(bottom=min(train_arr))

#         plt.show()
        
        pred = model.predict(torch.FloatTensor(X_test))
        
        pred = pred.exp().detach()     
        _, index = torch.max(pred,1)  
        pred = pred.numpy()
        index = index.numpy()
        
        kfold_scores.append(sklearn.metrics.accuracy_score(Y_test,index))
                
    return kfold_scores, val_arr, train_arr


In [143]:
def PCA_Classification(data, targets, param_list, filename, verbose=True, normalised=False):
    
    #set up pandas dataframe for result storage
    df = pd.DataFrame(
        {'Parameters':[],
         'CoDACl Scores':[],
         'CoDAPCA Scores':[],
         'CLRPCA Scores':[], 
         'PCA Scores':[],
         'Naive Scores':[],
         'CoDACl Val Loss':[], 
         'CoDACl Train Loss':[] 
        }).set_index('Parameters')
    
    
    df.to_csv(os.path.join(os.getcwd(),str(filename)+str(".csv")))

    
    for params in param_list:
        
        #TODO: run several times then average
        nn_dims = params[0]
        lr = params[1]
        lam = params[2]
        n_epochs = params[3]
        
        n_features = nn_dims[0]
        n_components = nn_dims[1]
        nn_shape= nn_dims[2]
        
        features = data[:, :n_features]
                
        if normalised == False:
            #normalise the compositional features
            features = np.array([feat/sum(feat) for feat in features])
            
        #encode targets
        le = LabelEncoder()
        le.fit(targets)
        targets = le.transform(targets)
        
        
        #keep folds constant across each algorithm for fair results
        kf = KFold(4)
        folds = [i for i in kf.split(features)] 


        #set up baseline algorithms:
        
        #CoDA-PCA:
        pca_coda = CodaPCA.NonParametricCodaPCA(n_components)
        Y_coda = pca_coda.fit_transform(np.array(features, dtype=np.float32))

        #CLR-PCA:
        pca_clr = CodaPCA.CLRPCA(n_components)
        Y_clr = pca_clr.fit_transform(np.array(features, dtype=np.float32))
       
        #PCA:
        sc = StandardScaler()
        features_ = sc.fit_transform(np.array(features, dtype=np.float32))
        pca_reg = decomposition.PCA(n_components)
        pca_reg.fit(features_)
        Y_pca = pca_reg.transform(features_)


        coda_score = enhanced_cross_val(Y_coda, np.array(targets), folds)
        clr_score = enhanced_cross_val(Y_clr, np.array(targets), folds) 
        pca_score = enhanced_cross_val(Y_pca, np.array(targets), folds)
        naive_score = enhanced_cross_val(features, np.array(targets), folds)
        
        
        codacl_score, val_arr, train_arr = coda_val(np.array(features,dtype=np.float32), np.array(targets), n_components, folds, nn_shape, lr, lam, n_epochs)

        
        #read/write each time to keep results if it crashes
        
        df = pd.read_csv(os.path.join(os.getcwd(),str(filename)+str(".csv"))).set_index('Parameters')
        df_row  = "nn {}, lr {}, lam {}, epochs {}, num_features {}, low_dimension {}".format(nn_shape, lr, lam, n_epochs, n_features, n_components)
        
        df.loc[df_row] = [codacl_score, coda_score, clr_score, pca_score, naive_score, val_arr, train_arr]
        
        df.to_csv(os.path.join(os.getcwd(),str(filename)+str(".csv")))


    
    if verbose:
        print("CoDA-PCA:")
        print(coda_score)
        print("CLR-PCA:")
        print(clr_score)
        print("PCA Classification:")
        print(pca_score)
        print ("Naive Classification:")
        print (naive_score)
        print ("Coda Cl:")
        print (codacl_score)

    return 


In [123]:
import pandas as pd
import numpy as np
rug_data = pd.read_hdf('rug.h5','rug')


#cut off data below certain threshold
n_features = 20

#go an extra 2 for the targets
data = np.array(rug_data)
np.random.shuffle(data)
targets = data[:,-2]

features = data[:,:-2]
len(targets)

282

In [119]:
#remove space added to some of the targets
for i in range(0, len(targets)):
    if targets[i] == 'Luing ':
        targets[i] = 'Luing'
    if targets[i] == 'Aax':
        targets[i] = 'AAx'

In [146]:
#experiments: should only need to specify the data and the params, then get a result file back
import itertools
#format [original size, low level dimension, [[encoder_shape], [decoder_shape]]
nn_dims = [[10,5, [[7,],[7,]]], [15,5, [[10,],[10,]]], [20,5, [[15,],[15,]]],[20,10, [[15,],[15,]]], [50,10, [[30,],[30,]]]]
lr_vals = [1e-3, 1e-4, 1e-5]
lam_vals = [1e-12,1e-11,1e-10,1e-8,1e-5]
epoch_vals = [100, 200,300, 400, 500, 1000]
param_list = list(itertools.product(*[nn_dims, lr_vals, lam_vals, epoch_vals]))
param_list

[([10, 5, [[7], [7]]], 0.001, 1e-12, 100),
 ([10, 5, [[7], [7]]], 0.001, 1e-12, 200),
 ([10, 5, [[7], [7]]], 0.001, 1e-12, 300),
 ([10, 5, [[7], [7]]], 0.001, 1e-12, 400),
 ([10, 5, [[7], [7]]], 0.001, 1e-12, 500),
 ([10, 5, [[7], [7]]], 0.001, 1e-12, 1000),
 ([10, 5, [[7], [7]]], 0.001, 1e-11, 100),
 ([10, 5, [[7], [7]]], 0.001, 1e-11, 200),
 ([10, 5, [[7], [7]]], 0.001, 1e-11, 300),
 ([10, 5, [[7], [7]]], 0.001, 1e-11, 400),
 ([10, 5, [[7], [7]]], 0.001, 1e-11, 500),
 ([10, 5, [[7], [7]]], 0.001, 1e-11, 1000),
 ([10, 5, [[7], [7]]], 0.001, 1e-10, 100),
 ([10, 5, [[7], [7]]], 0.001, 1e-10, 200),
 ([10, 5, [[7], [7]]], 0.001, 1e-10, 300),
 ([10, 5, [[7], [7]]], 0.001, 1e-10, 400),
 ([10, 5, [[7], [7]]], 0.001, 1e-10, 500),
 ([10, 5, [[7], [7]]], 0.001, 1e-10, 1000),
 ([10, 5, [[7], [7]]], 0.001, 1e-08, 100),
 ([10, 5, [[7], [7]]], 0.001, 1e-08, 200),
 ([10, 5, [[7], [7]]], 0.001, 1e-08, 300),
 ([10, 5, [[7], [7]]], 0.001, 1e-08, 400),
 ([10, 5, [[7], [7]]], 0.001, 1e-08, 500),
 ([10, 5

In [147]:
%matplotlib inline
PCA_Classification(features, targets, param_list,filename="class_test_1")

c:\users\u5801283\appdata\local\continuum\anaconda3\envs\honours\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
c:\users\u5801283\appdata\local\continuum\anaconda3\envs\honours\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
c:\users\u5801283\appdata\local\continuum\anaconda3\envs\honours\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


loading
loading
loading
loading


c:\users\u5801283\appdata\local\continuum\anaconda3\envs\honours\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
c:\users\u5801283\appdata\local\continuum\anaconda3\envs\honours\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
c:\users\u5801283\appdata\local\continuum\anaconda3\envs\honours\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


loading
loading
loading
loading


c:\users\u5801283\appdata\local\continuum\anaconda3\envs\honours\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
c:\users\u5801283\appdata\local\continuum\anaconda3\envs\honours\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
c:\users\u5801283\appdata\local\continuum\anaconda3\envs\honours\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


...

loading
loading
loading
loading


c:\users\u5801283\appdata\local\continuum\anaconda3\envs\honours\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
c:\users\u5801283\appdata\local\continuum\anaconda3\envs\honours\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
c:\users\u5801283\appdata\local\continuum\anaconda3\envs\honours\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
c:\users\u5801283\appdata\local\continuum\anaconda3\envs\honours\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
c:\users\u5801283\appdata\local\

loading
loading
loading
loading


c:\users\u5801283\appdata\local\continuum\anaconda3\envs\honours\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
c:\users\u5801283\appdata\local\continuum\anaconda3\envs\honours\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
c:\users\u5801283\appdata\local\continuum\anaconda3\envs\honours\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
c:\users\u5801283\appdata\local\continuum\anaconda3\envs\honours\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
c:\users\u5801283\appdata\local\

loading
loading
loading
loading
CoDA-PCA:
[0.4788732394366197, 0.30985915492957744, 0.4142857142857143, 0.34285714285714286]
CLR-PCA:
[0.352112676056338, 0.4507042253521127, 0.3, 0.42857142857142855]
PCA Classification:
[0.38028169014084506, 0.323943661971831, 0.2571428571428571, 0.4]
Naive Classification:
[0.30985915492957744, 0.29577464788732394, 0.2714285714285714, 0.2857142857142857]
Coda Cl:
[0.2676056338028169, 0.014084507042253521, 0.014285714285714285, 0.0]


[10, 5, [[7], [7]]]

In [67]:
df = pd.DataFrame(
    {'trial_num': [1, 2, 3, 1, 2, 3],
     'subject': [1, 1, 1, 2, 2, 2],
     'samples': [list(np.random.randn(3).round(2)) for i in range(6)]
    }).set_index("trial_num")


'trial_num,subject,samples\r\n1,1,"[0.93, -0.19, -1.06]"\r\n2,1,"[1.1, 0.77, -0.86]"\r\n3,1,"[2.16, -0.8, 1.23]"\r\n1,2,"[-2.13, -0.64, 0.16]"\r\n2,2,"[-0.13, -0.96, 0.7]"\r\n3,2,"[-0.37, 0.67, 0.89]"\r\n'

In [132]:
    #set up pandas dataframe for result storage
    df = pd.DataFrame(
        {'Parameters':[],
         'CoDACl Scores':[],
         'CoDAPCA Scores':[],
         'CLRPCA Scores':[], 
         'PCA Scores':[],
         'Naive Scores':[],
         'CoDACl Val Loss':[], 
         'CoDACl Train Loss':[] 
        }).set_index('Parameters')

    df.loc['asdfgasdasdasdasdasdasdasdasdasdasdasdsadasdsadasdasdasd'] = [[1,2,3], [1,2,3], [2,3,4], [23,4,5], [123,3,4], [1,2,3,4,5,6,7,8,9], [1,2,3,4,5,6,7,8,9]]
   
    df.to_csv(os.path.join(os.getcwd(),"test.csv"))
    x = pd.read_csv("test.csv")
    x

,Parameters,CoDACl Scores,CoDAPCA Scores,CLRPCA Scores,PCA Scores,Naive Scores,CoDACl Val Loss,CoDACl Train Loss
0,asdfgasdasdasdasdasdasdasdasdasdasdasdsadasdsa...,"[1, 2, 3]","[1, 2, 3]","[2, 3, 4]","[23, 4, 5]","[123, 3, 4]","[1, 2, 3, 4, 5, 6, 7, 8, 9]","[1, 2, 3, 4, 5, 6, 7, 8, 9]"
